# AFL Model - Part 4 - Weekly Predictions
Now that we have explored different algorithms for modelling, we can implement our chosen model and predict this week's AFL games! All you need to do is run the afl_modelling script each Thursday or Friday to predict the following week's games.

In [2]:
# Import Modules
from afl_feature_creation import prepare_afl_features
import afl_data_cleaning
import afl_feature_creation
import afl_modelling
import datetime
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

## Creating The Features For This Weekend's Games
To actually predict this weekend's games, we need to create the same features that we have created in the previous tutorials for the games that will be played this weekend. This includes all the rolling averages, efficiency features, elo features etc. So the majority of this tutorial will be using previously defined functions to create features for the following weekend's games.

### Create Next Week's DataFrame
Let's first get our cleaned afl_data dataset, as well as the odds for next weekend and the 2018 fixture.

In [3]:
# Grab the cleaned AFL dataset and the column order
afl_data = afl_data_cleaning.prepare_afl_data()
ordered_cols = afl_data.columns

# Define a function which grabs the odds for each game for the following weekend
def get_next_week_odds(path):
    # Get next week's odds
    next_week_odds = pd.read_csv(path)
    next_week_odds = next_week_odds.rename(columns={"team_1": "home_team", 
                                                "team_2": "away_team", 
                                                "team_1_odds": "odds", 
                                                "team_2_odds": "odds_away"
                                               })
    return next_week_odds

# Import the fixture
# Define a function which gets the fixture and cleans it up
def get_fixture(path):
    # Get the afl fixture
    fixture = pd.read_csv(path)

    # Replace team names and reformat
    fixture = fixture.replace({'Brisbane Lions': 'Brisbane', 'Footscray': 'Western Bulldogs'})
    fixture['Date'] = pd.to_datetime(fixture['Date']).dt.date.astype(str)
    fixture = fixture.rename(columns={"Home.Team": "home_team", "Away.Team": "away_team"})
    return fixture

next_week_odds = get_next_week_odds("data/weekly_odds.csv")
fixture = get_fixture("data/afl_fixture_2018.csv")

In [4]:
next_week_odds

,home_team,away_team,odds,odds_away
0,Adelaide,Geelong,2.16,1.86
1,St Kilda,Carlton,1.40,3.50
2,Hawthorn,Brisbane,1.27,4.80
3,Melbourne,Western Bulldogs,1.19,8.20
4,Gold Coast,Essendon,5.60,1.23
5,GWS,Richmond,2.84,1.55
6,Collingwood,West Coast,1.44,2.68
7,North Melbourne,Sydney,2.22,1.82
8,Fremantle,Port Adelaide,4.60,1.28


In [5]:
fixture.tail()

,Date,Season,Season.Game,Round,home_team,away_team,Venue
211,2018-08-24,2018,212,NaN,Brisbane,West Coast,Gabba
212,2018-08-24,2018,213,NaN,Carlton,St Kilda,Etihad Stadium
213,2018-08-24,2018,214,NaN,Richmond,Melbourne,MCG
214,2018-08-24,2018,215,NaN,Port Adelaide,Essendon,Adelaide Oval
215,2018-08-24,2018,216,NaN,Fremantle,Collingwood,Optus Stadium


Now that we have these DataFrames, we will define a function which combines the fixture and next week's odds to create a single DataFrame for the games over the next 7 days. To use this function we will need Game IDs for next week. So we will create another function which creates Game IDs by using the Game ID from the last game played and adding 1 to it.

In [6]:
# Define a function which creates game IDs for this week's footy games
def create_next_weeks_game_ids(afl_data):
    odds = get_next_week_odds("data/weekly_odds.csv")

    # Get last week's Game ID
    last_afl_data_game = afl_data['Game'].iloc[-1]

    # Create Game IDs for next week
    game_ids = [(i+1) + last_afl_data_game for i in range(odds.shape[0])]
    return game_ids

# Define a function which creates this week's footy game DataFrame
def get_next_week_df(afl_data):
    # Get the fixture and the odds for next week's footy games
    fixture = get_fixture("data/afl_fixture_2018.csv")
    next_week_odds = get_next_week_odds("data/weekly_odds.csv")
    next_week_odds['Game'] = create_next_weeks_game_ids(afl_data)

    # Get today's date and next week's date and create a DataFrame for next week's games
    todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
    date_in_7_days = (datetime.datetime.today() + datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    fixture = fixture[(fixture['Date'] >= todays_date) & (fixture['Date'] < date_in_7_days)].drop(columns=['Season', 'Season.Game', 'Round'])
    next_week_df = pd.merge(fixture, next_week_odds, on=['home_team', 'away_team'])

    # Split the DataFrame onto two rows for each game
    h_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds']]
    h_df['Team'] = h_df['home_team']
    h_df['Opposition'] = h_df['away_team']
    h_df['Home?'] = 1
    a_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds_away']].rename(columns={'odds_away': 'odds'})
    a_df['Team'] = a_df['away_team']
    a_df['Opposition'] = a_df['home_team']
    a_df['Home?'] = 0
    next_week = a_df.append(h_df).sort_values(by='Game')
    return next_week

In [7]:
next_week_df = get_next_week_df(afl_data)
game_ids_next_round = create_next_weeks_game_ids(afl_data)
next_week_df.head()

,Date,Game,home_team,away_team,odds,Team,Opposition,Home?
0,2018-07-13,15334,St Kilda,Carlton,3.50,Carlton,St Kilda,0
0,2018-07-13,15334,St Kilda,Carlton,1.40,St Kilda,Carlton,1
1,2018-07-14,15335,Hawthorn,Brisbane,4.80,Brisbane,Hawthorn,0
1,2018-07-14,15335,Hawthorn,Brisbane,1.27,Hawthorn,Brisbane,1
2,2018-07-14,15336,Melbourne,Western Bulldogs,8.20,Western Bulldogs,Melbourne,0


### Create Each Feature
Now let's append next week's DataFrame to our afl_data DataFrame and then create all the features we used in the [AFL Feature Creation Tutorial](0.2. afl_feature_creation_tutorial.ipynb).

In [8]:
# Append next week's games to our afl_data DataFrame
afl_data = afl_data.append(next_week_df).reset_index(drop=True)
afl_data = afl_data[ordered_cols]

# Create disposal efficiency column
afl_data['disposal_efficiency'] = afl_data['ED'] / afl_data['D']

# Create rolling averages for our statistics
cols_indx_start = afl_data.columns.get_loc("GA")
afl_avgs = afl_feature_creation.create_rolling_averages(afl_data, 6, afl_data.columns[cols_indx_start:])

# Create form between teams feature
afl_avgs = afl_feature_creation.form_between_teams(afl_avgs, 6)

# Apply elos
elos, probs, elo_dict = afl_feature_creation.elo_applier(afl_data, 24)
afl_avgs['home_elo'] = afl_avgs['Game'].map(elos).str[0]
afl_avgs['away_elo'] = afl_avgs['Game'].map(elos).str[1]

# Get elos for next week
afl_avgs.loc[afl_avgs['home_elo'].isna(), 'home_elo'] = afl_avgs[afl_avgs['home_elo'].isna()]['home_team'].map(elo_dict)
afl_avgs.loc[afl_avgs['away_elo'].isna(), 'away_elo'] = afl_avgs[afl_avgs['away_elo'].isna()]['away_team'].map(elo_dict)

# Create Adjusted Margin and then Average it over a 6 game window
afl_avgs = afl_feature_creation.map_elos(afl_avgs)
afl_avgs['Adj_elo_ave_margin'] = afl_avgs['Margin'] * afl_avgs['elo_Opp'] / afl_avgs['elo']
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Adj_elo_ave_margin'])

# Create average elo of opponents beaten/lost to feature
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='beaten')
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='lost')

# Create regular margin rolling average
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Margin'])
afl_avgs.tail()

,Team,odds,Date,home_team,away_team,Behinds,Game,Goals,Home?,Opposition,Opposition Behinds,Opposition Goals,Opposition Points,Points,Round,Venue,Season,Status,GA_ave_6,CP_ave_6,UP_ave_6,ED_ave_6,CM_ave_6,MI5_ave_6,One.Percenters_ave_6,BO_ave_6,K_ave_6,HB_ave_6,D_ave_6,M_ave_6,G_ave_6,B_ave_6,T_ave_6,HO_ave_6,I50_ave_6,CL_ave_6,CG_ave_6,R50_ave_6,FF_ave_6,FA_ave_6,AF_ave_6,SC_ave_6,CCL_ave_6,SCL_ave_6,SI_ave_6,MG_ave_6,TO_ave_6,ITC_ave_6,T5_ave_6,disposal_efficiency_ave_6,form_over_opposition_6,home_elo,away_elo,elo,elo_Opp,Adj_elo_ave_margin_ave_6,average_elo_opponents_beaten_6,average_elo_opponents_lost_6,Margin_ave_6
2955,Collingwood,1.44,2018-07-15,Collingwood,West Coast,NaN,15339,NaN,1,West Coast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.500000,145.333333,284.000000,320.666667,11.333333,11.500000,54.166667,8.166667,225.166667,201.000000,426.166667,106.666667,11.333333,9.166667,65.000000,46.000000,52.666667,34.500000,54.666667,40.166667,23.666667,18.166667,1749.833333,1763.166667,9.000000,25.500000,104.000000,5966.666667,74.500000,75.666667,9.500000,0.751951,3,1508.140954,1592.391714,1508.140954,1592.391714,7.255246,1435.947110,1613.567587,9.666667
2956,Sydney,1.82,2018-07-15,North Melbourne,Sydney,NaN,15340,NaN,0,North Melbourne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.333333,150.833333,225.166667,271.833333,11.333333,9.000000,48.333333,3.833333,213.000000,166.000000,379.000000,86.333333,11.500000,7.666667,63.666667,27.000000,52.666667,36.666667,55.833333,44.833333,20.833333,23.333333,1539.166667,1709.166667,11.666667,25.000000,72.166667,5914.833333,73.500000,74.333333,11.500000,0.715256,4,1472.869065,1662.752549,1662.752549,1472.869065,11.729715,1521.968557,1589.986226,13.500000
2957,North Melbourne,2.22,2018-07-15,North Melbourne,Sydney,NaN,15340,NaN,1,Sydney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.666667,154.666667,204.500000,246.166667,11.833333,10.000000,42.833333,4.000000,193.833333,159.500000,353.333333,77.000000,11.833333,8.666667,65.166667,44.833333,49.833333,35.833333,58.666667,34.666667,24.833333,20.000000,1481.500000,1620.000000,11.166667,24.666667,81.500000,5343.500000,73.500000,76.000000,10.666667,0.696222,2,1472.869065,1662.752549,1472.869065,1662.752549,5.552295,1520.030707,1443.574206,5.333333
2958,Port Adelaide,1.28,2018-07-15,Fremantle,Port Adelaide,NaN,15341,NaN,0,Fremantle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.166667,152.166667,226.333333,262.833333,12.833333,13.000000,64.166667,6.000000,217.666667,155.666667,373.333333,89.666667,12.666667,10.166667,77.166667,44.833333,59.166667,39.666667,57.666667,36.166667,23.166667,25.666667,1619.166667,1765.666667,12.666667,27.000000,94.666667,5807.666667,73.333333,74.333333,15.166667,0.702300,4,1426.443873,1583.704866,1583.704866,1426.443873,17.349483,1400.051819,1557.820007,20.333333
2959,Fremantle,4.60,2018-07-15,Fremantle,Port Adelaide,NaN,15341,NaN,1,Port Adelaide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.666667,133.166667,236.000000,267.833333,8.666667,11.833333,46.666667,2.833333,203.166667,158.333333,361.500000,93.500000,10.833333,6.833333,69.333333,44.666667,47.333333,33.333333,60.333333,38.500000,18.333333,21.666667,1553.833333,1611.166667,11.166667,22.166667,89.833333,5317.500000,71.166667,67.000000,9.833333,0.739750,2,1426.443873,1583.704866,1426.443873,1583.704866,-8.857551,1401.112651,1553.264182,-6.333333


Our DataFrame looks great! Although there are some NaNs, these are only in columns like 'Goals' which obviously don't have a value yet as the game hasn't been played.

Now we need to get our DataFrame on one line so that each row corresponds to one footy game. Let's use our previously defined function for this. Let's also drop the columns with NaNs which we don't need.

In [9]:
# Get each footy match on individual rows and drop irrelevant columns
one_line = afl_modelling.get_df_on_one_line(afl_avgs)

# Drop duplicate columns and unnecessary columns
cols_to_drop = ['Opposition Behinds', 'Goals', 'Behinds', 'Opposition Goals', 'Opposition Points', 'Points', 'Round', 'Venue', 'Season', 'Status',
               'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'elo', 'elo_Opp', 'Behinds_away',
               'Goals_away', 'home_elo_away', 'away_elo_away', 'elo_away', 'elo_Opp_away']
one_line = one_line.drop(columns=cols_to_drop)
# Drop all columns where home_elo or away_elo == 1500 exactly, as this is the first game played
one_line = one_line[(one_line['home_elo'] != 1500) & (one_line['away_elo'] != 1500)]

# Drop Na rows from calculating moving averages
one_line = one_line.dropna(axis=0)

Now let's create our differential_df and then filter out DataFrame to only include this weekend's game, based on the Game IDs we created earlier

In [10]:
diff_df = afl_modelling.get_diff_df(one_line)
diff_df = diff_df.drop(columns=['odds', 'odds_away']).select_dtypes(include=[np.number])
prediction_feature_set = diff_df[diff_df['Game'].isin(game_ids_next_round)].dropna(axis=1)

In [11]:
prediction_feature_set

,Game,home_elo,away_elo,CCL_ave_6_away,SCL_ave_6_away,SI_ave_6_away,MG_ave_6_away,TO_ave_6_away,ITC_ave_6_away,T5_ave_6_away,GA_ave_6_diff,CP_ave_6_diff,UP_ave_6_diff,ED_ave_6_diff,CM_ave_6_diff,MI5_ave_6_diff,One.Percenters_ave_6_diff,BO_ave_6_diff,K_ave_6_diff,HB_ave_6_diff,D_ave_6_diff,M_ave_6_diff,G_ave_6_diff,B_ave_6_diff,T_ave_6_diff,HO_ave_6_diff,I50_ave_6_diff,CL_ave_6_diff,CG_ave_6_diff,R50_ave_6_diff,FF_ave_6_diff,FA_ave_6_diff,AF_ave_6_diff,SC_ave_6_diff,disposal_efficiency_ave_6_diff,form_over_opposition_6_diff,Adj_elo_ave_margin_ave_6_diff,average_elo_opponents_beaten_6_diff,average_elo_opponents_lost_6_diff,Margin_ave_6_diff,implied_odds_prob,implied_odds_prob_away
1500,15334,1422.699826,1249.516972,11.166667,21.333333,74.166667,4784.666667,66.500000,63.833333,8.666667,2.333333,13.500000,31.833333,32.666667,0.166667,2.166667,2.833333,1.166667,10.000000,27.166667,37.166667,-0.333333,2.166667,2.000000,-1.166667,-3.333333,11.500000,1.666667,-2.166667,0.833333,-3.500000,-2.666667,94.833333,123.333333,0.013466,0,33.642420,-122.052952,113.809381,29.166667,0.714286,0.285714
1501,15335,1601.212897,1264.807130,11.500000,24.333333,98.166667,5655.166667,69.000000,68.833333,7.500000,-0.666667,5.166667,-6.666667,-4.333333,0.000000,-1.000000,-1.500000,4.166667,-6.833333,3.833333,-3.000000,-11.166667,1.000000,0.166667,17.666667,-4.166667,0.166667,0.000000,-3.833333,-7.000000,-1.500000,-0.333333,25.833333,23.166667,-0.003173,4,9.649333,46.605089,-96.940718,8.833333,0.787402,0.208333
1502,15336,1489.199372,1463.143986,10.333333,22.166667,73.166667,5386.500000,76.333333,77.833333,12.666667,4.000000,22.666667,8.000000,15.000000,4.833333,5.666667,-10.166667,-3.333333,11.000000,14.500000,25.500000,6.666667,6.166667,4.000000,6.333333,22.000000,11.500000,9.166667,3.500000,-9.333333,1.166667,0.500000,170.000000,181.333333,-0.010136,-2,71.766382,6.081699,73.047408,71.000000,0.840336,0.121951
1503,15337,1293.271340,1496.279191,9.666667,28.000000,86.166667,5640.000000,67.500000,66.666667,12.166667,-3.333333,7.166667,-50.333333,-54.500000,-4.333333,-4.000000,10.166667,-6.333333,-22.166667,-24.500000,-46.666667,-25.666667,-2.666667,-0.166667,9.333333,9.500000,1.000000,-0.833333,2.833333,2.000000,4.333333,-1.000000,-154.500000,-199.000000,-0.056577,-4,-37.810434,-260.538878,53.078812,-30.000000,0.178571,0.813008
1504,15338,1583.093084,1623.317097,13.333333,20.500000,97.500000,6019.000000,71.166667,80.666667,14.500000,-1.500000,1.333333,18.000000,18.833333,-2.833333,-1.166667,4.500000,-2.833333,-1.166667,20.333333,19.166667,-3.333333,-2.666667,2.833333,0.666667,3.666667,-6.666667,5.000000,-9.166667,4.000000,2.666667,-2.166667,29.500000,29.833333,0.014403,-2,-14.147148,-115.066184,-37.089721,-14.000000,0.352113,0.645161
1505,15339,1508.140954,1592.391714,12.500000,24.666667,89.166667,5560.833333,69.333333,70.333333,10.333333,0.166667,2.333333,80.166667,69.166667,-1.166667,1.500000,5.000000,3.166667,7.833333,73.333333,81.166667,13.666667,-0.666667,1.833333,2.833333,-4.333333,2.833333,-2.666667,3.166667,4.333333,-1.333333,0.166667,214.166667,128.333333,0.022468,0,-1.340832,-36.450426,-4.683358,1.166667,0.694444,0.373134
1506,15340,1472.869065,1662.752549,11.666667,25.000000,72.166667,5914.833333,73.500000,74.333333,11.500000,0.333333,3.833333,-20.666667,-25.666667,0.500000,1.000000,-5.500000,0.166667,-19.166667,-6.500000,-25.666667,-9.333333,0.333333,1.000000,1.500000,17.833333,-2.833333,-0.833333,2.833333,-10.166667,4.000000,-3.333333,-57.666667,-89.166667,-0.019034,-2,-6.177420,-1.937850,-146.412020,-8.166667,0.450450,0.549451
1507,15341,1426.443873,1583.704866,12.666667,27.000000,94.666667,5807.666667,73.333333,74.333333,15.166667,-1.500000,-19.000000,9.666667,5.000000,-4.166667,-1.166667,-17.500000,-3.166667,-14.500000,2.666667,-11.833333,3.833333,-1.833333,-3.333333,-7.833333,-0.166667,-11.833333,-6.333333,2.666667,2.333333,-4.833333,-4.000000,-65.333333,-154.500000,0.037450,-2,-26.207034,1.060831,-4.555825,-26.666667,0.217391,0.781250


Great! We now have our feature DataFrame for this weekend's games. Let's now model this to create our predictions

## Creating Our Predictions
To create our predictions we will use similar code to what we used in our AFL Modelling Tutorial. We will need to train our model on all the data we have (up until last week's games), and then use our trained model to predict this week. Again, we will use Stacking with XGB, like we did in our AFL Modelling Tutorial.

In [12]:
afl = prepare_afl_features(window=6, k_factor=24).dropna().sort_values(by='Game')
afl = afl_modelling.get_df_on_one_line(afl)

# Drop columns which leak data and which we don't need
dropped_cols = ['Behinds', 'Goals', 'Opposition Behinds', 'Opposition Goals', 'Opposition Points', 'Points',
               'elo', 'elo_Opp', 'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'home_win_away',
               'home_elo_away', 'away_elo_away']
afl = afl.drop(columns=dropped_cols)
# Get a differential DataFrame - subtracting the away features from the home features
diff_df = afl_modelling.get_diff_df(afl)

In [13]:
# Hard code estimators from our last tutorial
all_estimators = [
    LogisticRegression(C=0.75, class_weight=None, dual=False, fit_intercept=True,
       intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
       penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
       verbose=0, warm_start=False),
    
    RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
     max_iter=None, normalize=False, random_state=None, solver='auto',
     tol=0.001),
    
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
         max_depth=15, max_features='sqrt', max_leaf_nodes=None,
         min_impurity_decrease=0.0, min_impurity_split=None,
         min_samples_leaf=1, min_samples_split=10,
         min_weight_fraction_leaf=0.0, n_estimators=750, n_jobs=-1,
         oob_score=False, random_state=5, verbose=0, warm_start=False),
    
    GaussianNB(priors=None),
    
    LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
           solver='svd', store_covariance=False, tol=0.0001)
]

# Hard code best cols
best_cols = ['home_elo',
'away_elo',
'GA_ave_6_diff',
'CP_ave_6_diff',
'UP_ave_6_diff',
'ED_ave_6_diff',
'CM_ave_6_diff',
'MI5_ave_6_diff',
'One.Percenters_ave_6_diff',
'BO_ave_6_diff',
'HB_ave_6_diff',
'M_ave_6_diff',
'G_ave_6_diff',
'T_ave_6_diff',
'HO_ave_6_diff',
'I50_ave_6_diff',
'CL_ave_6_diff',
'CG_ave_6_diff',
'R50_ave_6_diff',
'FF_ave_6_diff',
'FA_ave_6_diff',
'AF_ave_6_diff',
'SC_ave_6_diff',
'disposal_efficiency_ave_6_diff',
'R50_efficiency_ave_6_diff',
'I50_efficiency_ave_6_diff',
'Adj_elo_ave_margin_ave_6_diff',
'average_elo_opponents_beaten_6_diff',
'average_elo_opponents_lost_6_diff',
'Margin_ave_6_diff',
'implied_odds_prob',
'implied_odds_prob_away']

In [14]:
# Create our train sets
X = diff_df.drop(columns=['home_win']).select_dtypes(include=[np.number])
y = diff_df['home_win']

features = prediction_feature_set.columns

# Predict Next Round
preds_next_round = afl_modelling.implement_xgb_stacking(X[features].drop(columns='Game'), y, 
                                                        prediction_feature_set.drop(columns='Game'), all_estimators)

C:\Users\wardj\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Now that we have our predictions, the final step is to put our predictions into a DataFrame so they're easy on the eye, and our predictions correspond to actual home and away teams rather than Game IDs. This is the final step.

In [15]:
preds_df = pd.DataFrame({
    "Game": prediction_feature_set['Game'],
    "Prediction (home_win)": preds_next_round
})

prediction_feature_set['home_odds'] = 1 / prediction_feature_set['implied_odds_prob']
prediction_feature_set['away_odds'] = 1 / prediction_feature_set['implied_odds_prob_away']

final_preds_df = pd.merge(preds_df, next_week_df[['home_team', 'away_team', 'Game']], on='Game').drop_duplicates()
final_preds_df = pd.merge(final_preds_df, prediction_feature_set[['home_odds', 'away_odds', 'home_elo', 'away_elo', 'Game']], on='Game')
final_preds_df['Predicted Winner'] = final_preds_df.apply(lambda x: x['home_team'] if x['Prediction (home_win)'] == 1 else x['away_team'], axis=1)

In [16]:
final_preds_df[['Game', 'home_team', 'away_team', 'Predicted Winner', 'home_odds', 'away_odds', 'home_elo', 'away_elo']]

,Game,home_team,away_team,Predicted Winner,home_odds,away_odds,home_elo,away_elo
0,15334,St Kilda,Carlton,St Kilda,1.40,3.50,1422.699826,1249.516972
1,15335,Hawthorn,Brisbane,Hawthorn,1.27,4.80,1601.212897,1264.807130
2,15336,Melbourne,Western Bulldogs,Melbourne,1.19,8.20,1489.199372,1463.143986
3,15337,Gold Coast,Essendon,Essendon,5.60,1.23,1293.271340,1496.279191
4,15338,GWS,Richmond,Richmond,2.84,1.55,1583.093084,1623.317097
5,15339,Collingwood,West Coast,Collingwood,1.44,2.68,1508.140954,1592.391714
6,15340,North Melbourne,Sydney,Sydney,2.22,1.82,1472.869065,1662.752549
7,15341,Fremantle,Port Adelaide,Port Adelaide,4.60,1.28,1426.443873,1583.704866


## Conclusion
Congratulations! You have created AFL predictions for this week. If you are beginner to this, don't be overwhelmed. The process gets easier each time you do it. And it is super rewarding. In future iterations we will update this tutorial to predict actual odds, and then integrate this with Betfair's API so that you can create an automated betting strategy using Machine Learning to create your predictions!